In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate('path/to/serviceAccount.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

def subscribe_user_to_package(user_id, package, start_date, payment_intent_id):
    user_ref = db.collection(u'users').document(user_id)
    subs = user_ref.collection(u'subscriptions').document()
    subs.set({
        'package': package,
        'start_date': start_date,
        'payment_status': 'pending',  # payment has not yet been confirmed
        'payment_intent': payment_intent_id,  # store the Stripe PaymentIntent ID
    })
    user_ref.update({
        'current_package': package,
        # reset remaining_analysis based on package
        'remaining_analysis': 50 if package == 'basic' else 100,  # assuming premium users get 100 analyses
    })

def log_payment(user_id, subscription_id, amount, date, status, payment_intent_id):
    payments_ref = db.collection(u'payments').document()
    payments_ref.set({
        'user_id': user_id,
        'subscription_id': subscription_id,
        'amount': amount,
        'date': date,
        'status': status,
        'payment_intent': payment_intent_id,  # store the Stripe PaymentIntent ID
    })

def use_analysis(user_id):
    user_ref = db.collection(u'users').document(user_id)
    user = user_ref.get()
    remaining = user.get('remaining_analysis')
    if remaining > 0:
        user_ref.update({
            'remaining_analysis': firestore.Increment(-1),
        })
        return True  # analysis was successful
    else:
        return False  # user is out of analysis


In [ ]:
def add_investigation(user_id, asins):
    user_ref = db.collection(u'users').document(user_id)
    user = user_ref.get()
    remaining = user.get('remaining_investigations')
    if remaining > 0:
        investigation_ref = db.collection(u'investigations').document()
        investigation_ref.set({
            'user_id': user_id,
            'timestamp': firestore.SERVER_TIMESTAMP,  # let Firestore generate a server-side timestamp
            'asins': asins,
        })
        user_ref.update({
            'remaining_investigations': firestore.Increment(-1),
        })
        return True  # investigation was successful
    else:
        return False  # user is out of investigations
}


In [ ]:
import stripe

stripe.api_key = 'your_stripe_api_key'

def create_payment(user_id, package, amount):
    payment_intent = stripe.PaymentIntent.create(
        amount=amount,  # amount in cents
        currency='usd',
        metadata={'user_id': user_id, 'package': package},
    )
    return payment_intent.client_secret


In [ ]:
from flask import Flask, request
import stripe

app = Flask(__name__)

stripe.api_key = 'your_stripe_api_key'

@app.route('/webhook', methods=['POST'])
def webhook_received():
    event = stripe.Webhook.construct_event(
        request.data, request.headers.get('stripe-signature'), 'your_stripe_webhook_secret'
    )

    if event['type'] == 'payment_intent.succeeded':
        payment_intent = event['data']['object']
        user_id = payment_intent['metadata']['user_id']
        package = payment_intent['metadata']['package']
        log_payment(user_id, package, payment_intent['amount'], payment_intent['created'], 'completed')

    return '', 200


In [ ]:
B0BCTTCSBZ
B0BLNBS36G
B0BW8Y2B8Z
B08X2324ZL
B09NR5JW8Y
B091325ZMB